In [1]:
#!pip install neattext
#!pip install -U spacy
#!pip install transformers

In [2]:
import spacy
import re
import nltk
import string
import sklearn
import neattext as nt
import neattext.functions as nfx
import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import AutoTokenizer, TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tanriverdi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\Users\Tanriverdi\anaconda3\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tanriverdi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [3]:
tweets_d = pd.read_csv('Data/tweets_disc_cleaned.csv')
tweets_c = pd.read_csv('Data/tweets_ctrl_cleaned.csv')
data = pd.concat([tweets_d])
data

,date,id,content,likes,retweets,disc_cat,Unnamed: 0.1,disc_cat_num
0,28 February 2023,1630587451436683264,bitch lmfaooooo gone rock sock ass,0,0,gender,NaN,2
1,28 February 2023,1630587450710958081,nvr vw person simpli cost repair hassl get don...,0,0,gender,NaN,2
2,28 February 2023,1630587450673319940,hate wen bitch say stop act care df said act l...,0,0,gender,NaN,2
3,28 February 2023,1630587450547634179,dnc pay well ask brooklyn dad hes lucki get pe...,0,0,gender,NaN,2
4,28 February 2023,1630587449503264769,sinong bitch ba yung hindi marunong mag sorri ...,0,0,gender,NaN,2
...,...,...,...,...,...,...,...,...
210913,28 February 2023,1630401406883561472,NaN,3,0,mental_health,105706.0,4
210914,28 February 2023,1630401254798118913,make sens,1,0,mental_health,105707.0,4
210915,28 February 2023,1630401175995527169,mean make sens miz host,0,0,mental_health,105708.0,4
210916,28 February 2023,1630400994222948352,disrespect key lime pieoh use lubric,1,0,mental_health,105709.0,4


In [4]:
print(data.dtypes)

date             object
id                int64
content          object
likes             int64
retweets          int64
disc_cat         object
Unnamed: 0.1    float64
disc_cat_num      int64
dtype: object


In [5]:
data = data.drop(columns=['id', 'Unnamed: 0.1', 'date'])

In [6]:
data

,content,likes,retweets,disc_cat,disc_cat_num
0,bitch lmfaooooo gone rock sock ass,0,0,gender,2
1,nvr vw person simpli cost repair hassl get don...,0,0,gender,2
2,hate wen bitch say stop act care df said act l...,0,0,gender,2
3,dnc pay well ask brooklyn dad hes lucki get pe...,0,0,gender,2
4,sinong bitch ba yung hindi marunong mag sorri ...,0,0,gender,2
...,...,...,...,...,...
210913,NaN,3,0,mental_health,4
210914,make sens,1,0,mental_health,4
210915,mean make sens miz host,0,0,mental_health,4
210916,disrespect key lime pieoh use lubric,1,0,mental_health,4


In [8]:
data.groupby('disc_cat').describe()


likes                                                     \
                 count      mean         std  min  25%  50%  75%      max   
disc_cat                                                                    
gender         54108.0  6.362664  352.636012  0.0  0.0  0.0  1.0  78072.0   
health         13026.0  5.044987   79.589581  0.0  0.0  0.0  1.0   5491.0   
mental_health  44088.0  4.131079  124.139970  0.0  0.0  0.0  1.0  13713.0   
race           35569.0  7.245607  434.912340  0.0  0.0  0.0  1.0  45986.0   
sex            64127.0  3.932665   55.210575  0.0  0.0  0.0  1.0   6547.0   

              retweets            ...               disc_cat_num            \
                 count      mean  ...  75%      max        count mean  std   
disc_cat                          ...                                        
gender         54108.0  1.069398  ...  0.0  11479.0      54108.0  2.0  0.0   
health         13026.0  0.752956  ...  0.0   1926.0      13026.0  5.0  0.0   
mental_health  44088.0  0.529078  ...  0.0   2694.0      44088.0  4.0  0.0   
race           35569.0  1.060024  ...  0.0   6789.0      35569.0  3.0  0.0   
sex            64127.0  0.761707  ...  0.0   3047.0      64127.0  1.0  0.0   

                                        
               min  25%  50%  75%  max  
disc_cat                                
gender         2.0  2.0  2.0  2.0  2.0  
health         5.0  5.0  5.0  5.0  5.0  
mental_health  4.0  4.0  4.0  4.0  4.0  
race           3.0  3.0  3.0  3.0  3.0  
sex            1.0  1.0  1.0  1.0  1.0  

[5 rows x 24 columns]

In [9]:
data['disc_cat'].value_counts()


sex              64127
gender           54108
mental_health    44088
race             35569
health           13026
Name: disc_cat, dtype: int64

In [11]:
data.isnull().sum()

content         20685
likes               0
retweets            0
disc_cat            0
disc_cat_num        0
dtype: int64

In [14]:
data.dropna(subset=['content'], inplace=True)

In [15]:
data.isnull().sum()

content         0
likes           0
retweets        0
disc_cat        0
disc_cat_num    0
dtype: int64

In [16]:
data.drop_duplicates(inplace=True)

In [17]:
data

,content,likes,retweets,disc_cat,disc_cat_num
0,bitch lmfaooooo gone rock sock ass,0,0,gender,2
1,nvr vw person simpli cost repair hassl get don...,0,0,gender,2
2,hate wen bitch say stop act care df said act l...,0,0,gender,2
3,dnc pay well ask brooklyn dad hes lucki get pe...,0,0,gender,2
4,sinong bitch ba yung hindi marunong mag sorri ...,0,0,gender,2
...,...,...,...,...,...
210814,truth exist regardless recognit,1,0,mental_health,4
210820,clinton took democrat parti far right terrain ...,6,1,mental_health,4
210835,think would happenh throw bone,2,0,mental_health,4
210844,fox doesnt tell want hear want believ,29,3,mental_health,4


In [20]:
data_train, data_test = train_test_split(data, test_size = 0.3, random_state = 42, shuffle = True, stratify = data.disc_cat_num)

In [22]:
data_train['disc_cat_num'].unique()


array([4, 2, 3, 1, 5], dtype=int64)

In [24]:
x_train = tokenizer(
    text = data_train['content'].tolist(),
    add_special_tokens = True,
    max_length = 100,
    truncation = True,
    padding = True,
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [25]:
max_len = 100


input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
input_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

embeddings = bert(input_ids, attention_mask=input_mask)[0]
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation='relu')(out)

y = Dense(6, activation='sigmoid')(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [29]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
initial_lr = 5e-5
decay_rate = 0.01
decay_steps = 1000
epsilon=1e-08
clipnorm=1.0

lr_schedule = ExponentialDecay(initial_lr, decay_steps, decay_rate, staircase=True)

optimizer = Adam(
    learning_rate=lr_schedule,
    epsilon=epsilon,
    clipnorm=clipnorm
)

loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('balanced_accuracy')

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric
)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                           

In [31]:
x_test = tokenizer(
    text = data_test['content'].tolist(), 
    add_special_tokens = True,
    max_length = 100,
    truncation = True,
    padding = True,
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [34]:
bert_train = model.fit(
    x={'input_ids':x_train['input_ids'], 'attention_mask':x_train['attention_mask']},
    y=to_categorical(data_train.disc_cat_num),
    validation_data=(
        {'input_ids':x_test['input_ids'], 'attention_mask':x_test['attention_mask']},to_categorical(data_test.disc_cat_num)
    ),
    epochs=1,
    batch_size=36
)

C:\Users\Tanriverdi\anaconda3\lib\site-packages\keras\backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


3158/3158 [==============================] - 24605s 8s/step - loss: 0.7518 - balanced_accuracy: 0.6856 - val_loss: 0.6720 - val_balanced_accuracy: 0.7182


In [35]:
# To use bert model later, we need to save both the weights and the model architecture and load them both. 
# We already saved the model architechture above when we downloaded the model.


model.save_weights('model_disc_det.h5')

# To use the mode again, load it into memory
# model.load_weights('model_disc_det.h5')

In [36]:
pred_raw = model.predict({'input_ids':x_test['input_ids'], 'attention_mask':x_test['attention_mask']})

1523/1523 [==============================] - 2899s 2s/step


In [37]:
y_pred = np.argmax(pred_raw, axis=1)

In [38]:
data_test.disc_cat_num

3537      1
44655     2
171177    3
134853    1
27926     1
         ..
46972     2
74466     3
6603      2
78559     4
85784     5
Name: disc_cat_num, Length: 48714, dtype: int64

In [40]:
print(classification_report(data_test.disc_cat_num, y_pred))

              precision    recall  f1-score   support

           1       0.73      0.76      0.74     14557
           2       0.56      0.86      0.68     12782
           3       0.86      0.46      0.60      7600
           4       0.94      0.69      0.79     10411
           5       0.97      0.70      0.81      3364

    accuracy                           0.72     48714
   macro avg       0.81      0.69      0.73     48714
weighted avg       0.77      0.72      0.72     48714

